In [1]:
#CHANGE THE PATH TO THE PATH AT YOUR PC!
path = "C:\\Users\\user\\Desktop\\TUe\\Topological\\Project\\Geolife Trajectories 1.3\\Data\\"
CITY_BORDERS = [40.19, 39.65, 115.98, 116.74]
#Optimal threshold for # of occurances per cell to be classified as a vetex - 600

import pandas as pd
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(".\\presets\\very_short.csv")

In [3]:
df

,latitude,longitude,altitude,date,trajectory,user,transportation
0,39.994622,116.326757,150,2008-10-27 19:54:52,0.200810,0,NaN
1,39.994614,116.326751,150,2008-10-27 19:54:54,0.200810,0,NaN
2,39.994602,116.326769,150,2008-10-27 19:54:59,0.200810,0,NaN
3,39.994599,116.326769,150,2008-10-27 19:55:04,0.200810,0,NaN
4,39.994599,116.326769,150,2008-10-27 19:55:07,0.200810,0,NaN
...,...,...,...,...,...,...,...
166955,39.986867,116.304267,65,2008-03-16 10:12:08,99.200803,99,NaN
166956,39.987133,116.305183,42,2008-03-16 10:13:19,99.200803,99,NaN
166957,39.987633,116.305267,90,2008-03-16 10:14:26,99.200803,99,NaN
166958,39.988183,116.306317,37,2008-03-16 10:16:10,99.200803,99,NaN


## This will be an attempt at the graphification - probably I won't do this in one go

In [4]:
[(f, b) for f, b in zip(range(6), range(5))]

[(0, 0), (1, 1), (2, 2), (3, 3), (4, 4)]

In [5]:
class Cell():
    '''Just your regular Square kind of a class'''
    def __init__(self, top: float, left: float, bottom: float, right: float):
        self.top = top
        self.left = left
        self.bottom = bottom
        self.right = right
        self.number_of_locations = 0
        self.is_a_vertex = False
        
    def __str__(self):
        return "Cell: top = " + str(round(self.top, 5)) + ", left = " + str(round(self.left, 5)) + \
    ", bottom = " + str(round(self.bottom, 5)) + ", right = " + str(round(self.right, 5))

    def belongs(self, latitude: float, longitude: float):
        '''checks if a point belongs to the cell - pass the point as (y, x)'''
        return (latitude > self.bottom and latitude <= self.top and longitude < self.right and longitude >= self.left)
    
    def vertexize(self, threshold = 600):
        '''makes the Cell become a vertex if has more locations that threshold'''
        
        if self.number_of_locations > threshold:
            self.is_a_vertex = True

    

class Grid():
    '''pretty much just a list of Cells that is available at self.cell_list'''
    
    def __init__(self, latitude_cells_nr: int, longitude_cells_nr: int, city_borders = CITY_BORDERS):
        self.latitude_cells_nr = latitude_cells_nr
        self.longitude_cells_nr = longitude_cells_nr
        self.city_borders = city_borders
        
        self.latitude_cell_length = round((city_borders[0] - city_borders[1])/latitude_cells_nr, 5)
        self.longitude_cell_length = round((city_borders[3] - city_borders[2])/longitude_cells_nr, 5)
        
        self.cell_list = []
        for i in range(latitude_cells_nr):
            for j in range(longitude_cells_nr):
                self.cell_list.append(Cell(top=city_borders[1] + self.latitude_cell_length*i,
                                     left=city_borders[2] + self.longitude_cell_length*j,
                                     bottom=city_borders[1] + self.latitude_cell_length*(i+1),
                                     right=city_borders[2] + self.longitude_cell_length*(j+1)))
                
    def __str__(self):
        return "Grid of side length (latitude) " + str(self.latitude_cell_length) + \
    " and (longitude) " + str(self.longitude_cell_length) + " and total number of cells: " + \
    str(self.latitude_cells_nr*self.longitude_cells_nr)

        
    def feed_list_of_points(self, points: list):
        '''point is a tuple (y, x) - DO NOT MESS THE Y AND X UP!
        increase the number_of_locations for each cell, if the point belongs there'''
        
        for point in points:
            for cell in self.cell_list:
                if cell.belongs(point[0], point[1]):
                    cell.number_of_locations += 1
                    break
                    
                    
    def feed_dataframe_of_points(self, df: pd.DataFrame):
        '''point is a row with "latitude" as y and "longitude" as x - DO NOT MESS Y AND X UP!
        increase the number_of_locations for each cell, if the point belongs there'''
        
        #iterating over DataFrame's rows...unfortunately
        for i in range(df.shape[0]):
            
            #printing to see the progress:
            if i % 25000 == 0:
                print(i)
                
            for cell in self.cell_list:
                if cell.belongs(df.loc[i]["latitude"], df.loc[i]["longitude"]):
                    cell.number_of_locations += 1
                    break
                    
    def feed_list_of_cell_numbers(self, cell_numbers: list):
        '''for each cell number in the list, update the corresponding cell details'''
        
        for number in cell_numbers:
            self.cell_list[int(number)].number_of_locations += 1
            
    
    def vertexize(self, threshold):
        '''runs vertexise on each sell of the grid'''
        
        for cell in self.cell_list:
            cell.vertexize(threshold)

In [6]:
grid = Grid(20, 20) 
print(grid)

Grid of side length (latitude) 0.027 and (longitude) 0.038 and total number of cells: 400


In [7]:
print(grid.cell_list[0].number_of_locations)

0


In [8]:
#grid.feed_dataframe_of_points(df) #Takes WAY too fucking long

In [9]:
def row_to_grid_latitude(latitude: float, latitude_cells_number: int, latitude_cell_length: float, CITY_BORDERS: list):
    '''For a given latitude in the row of a datafame, checks which column number in the grid the point belongs to'''
    
    for i in range(latitude_cells_number):
        if latitude < CITY_BORDERS[1] + i*latitude_cell_length: #we do not have to check the other containment!
            return i
    

def row_to_grid_longitude(longitude: float, longitude_cells_number: int, longitude_cell_length: float, CITY_BORDERS: list):
    '''For a given latitude in the row of a datafame, checks which column number in the grid the point belongs to'''
    
    for i in range(longitude_cells_number):
        if longitude < CITY_BORDERS[2] + i*longitude_cell_length: #we do not have to check the other containment!
            return i
        

def grid_row_and_column_to_cell_number(row_nr: int, column_nr: int, longitude_cells_nr: int):
    '''given the x and y in the grid, calculates the cell number'''
    
    return row_nr*longitude_cells_nr + column_nr

In [10]:
df['column_number_in_grid'] = df['latitude'].apply(row_to_grid_latitude, args=(grid.latitude_cells_nr, 
                                                                               grid.latitude_cell_length, 
                                                                               CITY_BORDERS))

df['row_number_in_grid'] = df['longitude'].apply(row_to_grid_longitude, args=(grid.longitude_cells_nr, 
                                                                               grid.longitude_cell_length, 
                                                                               CITY_BORDERS))

df['cell_number'] = df.apply(lambda x: grid_row_and_column_to_cell_number(x['column_number_in_grid'],
                                                                         x['row_number_in_grid'],
                                                                         grid.longitude_cells_nr),axis = 1)


df = df.drop(['column_number_in_grid', 'row_number_in_grid'], axis=0).dropna()

KeyError: "['column_number_in_grid' 'row_number_in_grid'] not found in axis"

In [11]:
grid.feed_list_of_cell_numbers(list(df.cell_number))

ValueError: cannot convert float NaN to integer

In [ ]:
wow = pd.Series([grid.cell_list[i].number_of_locations for i in range(grid.latitude_cells_nr*grid.longitude_cells_nr)])
wow = pd.Series([wow[i] for i in range(len(wow)) if wow[i] < 1000])

In [ ]:
wow.plot(kind='hist', bins=20)

In [ ]:
grid.vertexize(400)